In [2]:
# ライブラリのインポート
from bs4 import BeautifulSoup
import re
import requests
from time import sleep
from tqdm import tqdm
import pandas as pd# ライブラリのインポート
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd
from tqdm import tqdm


In [3]:
# SUUMOから物件情報を取得する関数
def scrape_suumo(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    contents = soup.find_all('div', class_='cassetteitem')
    data_list = []

    for content in contents:
        detail = content.find('div', class_='cassetteitem-detail')
        table = content.find('table', class_='cassetteitem_other')

        title = detail.find('div', class_='cassetteitem_content-title').text
        address = detail.find('li', class_='cassetteitem_detail-col1').text
        access = detail.find('li', class_='cassetteitem_detail-col2').text
        age_text = detail.find('li', class_='cassetteitem_detail-col3').text
        age_match = re.search(r'\d+', age_text)
        age = int(age_match.group()) if age_match else None

        tr_tags = table.find_all('tr', class_='js-cassette_link')

        for tr_tag in tr_tags:
            floor_text = re.search(r'\d+', tr_tag.find_all("td")[2].text)
            floor = int(floor_text.group()) if floor_text else None

            price, first_fee, capacity = tr_tag.find_all("td")[3:6]
            fee, management_fee = price.find_all('li')
            deposit, qratuity = first_fee.find_all('li')
            madori, menseki = capacity.find_all('li')

            cost_text = re.search(r'\d+', fee.text)
            cost = int(cost_text.group()) if cost_text else None

            size_text = re.search(r'\d+', menseki.text)
            size = int(size_text.group()) if size_text else None

            d = {
                "title": title,
                "address": address,
                "acess": access,
                "age": age,
                "floor": floor,
                "cost": cost,
                "management_fee": management_fee.text,
                "deposit": deposit.text,
                "qratuity": qratuity.text,
                "rayout": madori.text,
                "size": size,
            }

            data_list.append(d)

    return data_list

# メインの処理
url_template = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&sc=13102&sc=13103&sc=13104&sc=13113&ta=13&cb=0.0&ct=30.0&md=06&md=07&md=09&md=10&md=11&md=13&md=14&ts=1&et=9999999&mb=0&mt=9999999&cn=9999999&fw2='
last_page = int(BeautifulSoup(requests.get(url_template.format(1)).text, 'html.parser').find('ol', class_='pagination-parts').find_all('li')[-1].text)

# 全ページのデータを取得
all_data = []
for i in tqdm(range(1, last_page+1)):
    target_url = url_template.format(i)
    data_list = scrape_suumo(target_url)
    all_data.extend(data_list)
    sleep(1)
# データをデータフレームに変換
df = pd.DataFrame(all_data)
# all_dataからDataFrameを作成
df_all_data = pd.DataFrame(all_data)

# address、age、floor、costが同じ場合に重複とみなし、重複を排除
df_no_duplicates = df_all_data.drop_duplicates(subset=['address', 'age', 'floor', 'cost'])

print(df_no_duplicates)

100%|██████████| 57/57 [02:49<00:00,  2.97s/it]

                   title       address  \
0               マンション勝どき    東京都中央区勝どき２   
1               マンション勝どき    東京都中央区勝どき２   
2    ＪＲ総武線 東中野駅 3階建 築22年    東京都新宿区北新宿３   
3             銀座レジデンス伍番館      東京都中央区湊２   
4             銀座レジデンス伍番館      東京都中央区湊２   
..                   ...           ...   
135               コ-ト355    東京都新宿区西落合２   
136               コ-ト355    東京都新宿区西落合２   
137             レクシード神楽坂    東京都新宿区東五軒町   
138             レクシード神楽坂    東京都新宿区東五軒町   
971           セザールスカイリバー  東京都中央区日本橋箱崎町   

                                                 acess  age  floor  cost  \
0    \n都営大江戸線/勝どき駅 歩1分\n東京メトロ有楽町線/月島駅 歩8分\n東京メトロ日比谷...   43     11    19   
1    \n都営大江戸線/勝どき駅 歩1分\n東京メトロ有楽町線/月島駅 歩8分\n東京メトロ日比谷...   43     11    20   
2    \nＪＲ総武線/東中野駅 歩9分\n都営大江戸線/東中野駅 歩9分\nＪＲ総武線/大久保駅 ...   22      1    19   
3    \n東京メトロ有楽町線/新富町駅 歩6分\n東京メトロ日比谷線/八丁堀駅 歩7分\n都営大江...   20      2    19   
4    \n東京メトロ有楽町線/新富町駅 歩6分\n東京メトロ日比谷線/八丁堀駅 歩7分\n都営大江...   20     11    21   
..                                     

In [4]:
df_no_duplicates.to_csv('output_no_duplicates3.csv', index=False, encoding='utf-8-sig')

PermissionError: [Errno 13] Permission denied: 'output_no_duplicates3.csv'

In [6]:
# SQLite3データベースに接続
conn = sqlite3.connect('suumo_property_data_no_duplicates.db')

# df_no_duplicatesをSQLite3に保存
df_no_duplicates.to_sql('properties_no_duplicates', conn, index=False, if_exists='replace')

# データベース接続を閉じる
conn.close()


In [15]:
import streamlit as st
import sqlite3
import pandas as pd

# SQLite3データベースに接続
conn = sqlite3.connect('suumo_property_data_no_duplicates.db')


# データベースからデータを読み込む
query = "SELECT * FROM properties_no_duplicates"
df_no_duplicates = pd.read_sql(query, conn)


# データベース接続を閉じる
conn.close()



    

In [16]:

# Streamlitアプリの実装
def main():
    st.title('SUUMO Property Search App (No Duplicates)')

    # データを表示
    st.dataframe(df_no_duplicates)

    # データ検索の実装
    st.header('Search Properties')
    address_input = st.text_input('Enter Address:', '')
    age_input = st.number_input('Enter Age:', min_value=0)
    floor_input = st.number_input('Enter Floor:', min_value=0)
    cost_input = st.number_input('Enter Cost:', min_value=0)

    # 検索条件に基づいて結果を表示
    if st.button('Search'):
        filtered_df = filter_data(df_no_duplicates, address_input, age_input, floor_input, cost_input)
        st.dataframe(filtered_df)

def filter_data(df, address, age, floor, cost):
    # 検索条件に基づいてデータを絞り込む
    filtered_df = df
    if address:
        filtered_df = filtered_df[filtered_df['address'].str.contains(address)]
    if age is not None:
        filtered_df = filtered_df[filtered_df['age'] == age]
    if floor is not None:
        filtered_df = filtered_df[filtered_df['floor'] == floor]
    if cost is not None:
        filtered_df = filtered_df[filtered_df['cost'] == cost]

    return filtered_df

if __name__ == '__main__':
    main()

2024-01-19 00:12:15.974 
  command:

    streamlit run c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [17]:
jupyter nbconvert --to script your_notebook.ipynb


SyntaxError: invalid syntax (2205563367.py, line 1)